# Chromaticity Matrix calculations notebook

## How to use this notebook

The functions in here are adapted from the colour maths documented by Bruce Lindbloom here:
* http://www.brucelindbloom.com/Eqn_RGB_XYZ_Matrix.html

For colour space in the notebook, you need to find the (x,y) chromaticities (in CIE xyY colour space) of the three primaries (Red, Green, Blue) as well as the white point.  With these "source" and "destination" matrices can be calculated.

To run these notebooks, you'll need a recent Python 3 installed, and the Python 3 libraries "jupyter", "numpy", "csv" and "Pillow" (fork of "PIL"). 

This repo contains the most recently created CSV files from this notebook.  If you just want to take that data and use it, you don't need to run this notebook, and can just read it online in read-only if interested. 

## The generated output

This notebook will generate both matrices within the notebook document itself, and also write out a CSV file with the following items, left to right: 
* col_id - unique ID for use programatically
* Colour space description - description (unused, for human reference only)
* EOFT ("gamma") - description (unused, for human reference only)
* White point, Red, Green and Blue (x,y) chromaticities
  * If this is an official colour space, it will be taken from the exac co-ordinates of the spec.
  * If this is a measured colour space, it will be referenced as such.
  * If this is otherwise desired, it will be calculated using Bruce Lindbloom's "T to xy" formula (if "T" is a CIE standard D illuminant, it uses the 2018-corrected Planck Constant c2. This is not the case for other illuminants like CIE standard C illuminant)
    * http://www.brucelindbloom.com/Eqn_T_to_xy.html
* The 3x3 "source" matrix "Msrc", listed as 9 values - Msrc[0][0], Msrc[0][1] ... Msrc[2][2]
* The 3x3 "destination" matrix "Mdst", listed as 9 values - Mdst[0][0], Mdst[0][1] ... Mdst[2][2]
  * Note that these will be left out for most results, other than common modern display types
  * This is done intentionally to help make the process clearer.

## How to use the matrices:

The order in which you apply these is critical, as is the understanding of what you're trying to achieve and technical knowledge of your input source and final display device. 

Included in this repo is a "RECOMMENDATIONS.md" file, which will include a table of recommended settings for different sources and destinations. 

1) Capture/read your pixel data (the "source") as digital RGB triplet.  This is assumed to be in full range RGB format, so if it's in something else (YUV, YPbPr, YIQ, whatever), convert it to RGB.
2) Remove the EOTF ("gamma") from the input image. This needs to match your source. For example, sRGB uses a non-linear gamma, not a pure 2.2 for the whole section as some equations approximate. Early Apple Macs (pre-iMac) used gamma 1.8.  BT-709 assumes BT-1886 for gamma, which provides two different equations, although some content is lazily mastered to 2.2.  I won't specify the gamma functions, as the user will need to apply those.  However the resulting RGB triplet MUST be in linear RGB with no EOTF/gamma.
3) Normalise the RGB to the range 0.0-1.0. For full range 8bit sRGB that's as simple as dividing by 255 (if not already normalised).  For other colour spaces, you will need to compensate for limited range scaling (i.e.: 16-235 or 16-255 (depending on region) instead of 0-255), different sample/bit depths, etc.
4) Multiply the SOURCE matrix and linear RGB matrix.  This source matrix needs to match the W, R, G, B chromaticities of what you assume your source content to be.  Think of this as how you intended to view the content if it was on an era-matching display.  For example, if it was a Japanese Super Famicom game, you might want this to have "NTSC-J ARIB" chromaticities with a D93 white point, so you'd use that source matrix. This might seem counter-intuitive at first (as we're applying it on input), but this allows us to easily define the display characteristics of what we want the input image to be defined by. 

<code>
|X|          |R| 
|Y| = |Msrc| |G|
|Z|          |B|
</code>

6) You'll end up with a co-ordinate in CIE 1931 XYZ colour space.  In this colour space you can trivially do any other maths/matrix/linear-algebra/scaling functions you want.  Here is a good place to do things like apply further shaders, shadow masks, or other colour manipulations.  This colour space is device-independent, normalised, all-positive colour space designed mostly for doing mathematical operations.  However if you don't want to do any further transformations, move on to the next step.
7) Multiply the DESTINATION matrix and XYZ matrix. This is the display device ("destination") you're going to display the image on to the viewer, and the display matrix should match those settings.  So for example, if this is a modern PC monitor, you want this to be sRGB D65.  If this is a HD TV, BT-709 D65.  If this is a UHD ("4K") TV, BT-2020 D65.  The source matrix covered the conversion of the expected/assumed/intended image white points and RGB chromaticities (say, on our hypothetical Super Famicom displaying a Japanese game in assumed "NTSC-J ARIB" D93) into XYZ. The destination matrix is how we want to render those colours taken from the input out to a specific display device/monitor/TV. You should almost never want to use a destination matrix of D93 (that should be the source matrix instead, if the content was originally intended to be D93).

<code>
|R|          |X| 
|G| = |Mdst| |Y|
|B|          |Z|
</code>

8) You now have your correct linear RGB co-ordinates to display.  From here, you want to apply your destination EOTF to match that expected by your display, and/or do any conversions out to other display colour spaces like YUV / YPbPr, etc. And again, ensure to scale to limited range if required.

## See it in action

The notebook "Colourspace_read_write_tests.ipynb" includes a test of this process, taking in an sRGB image where the rendered white point was D65, however we wanted to convert it to D93.  As such we applied the sRGB D93 source matrix (after removing gamma), and then applied the sRGB D65 destination matrix.  The resulting output shows a modified image which, when viewed on an sRGB D65 PC monitor, has the appearance of an image as if it was actually viewed on a D93 screen instead.  

This same effect can be achieved with a Chromatic Adaptation matrix (see "Chromatic_Adaptation_tests.ipynb" for a working example).  However that's more useful for still image conversion, and cannot compensate for RGB chromaticities, only white point adjustments.  For things like capture devices or internal emulator code, or where a user specifically wants to simulate the given RGB chromaticities of a specific display (for example, "Japanese phosphors" on certain CRTs that differ to the RGB chromaticities on western CRTs and/or specifications of modern LCD/OLED monitors and TVs), the method outlined in this notebook with the supplied matrices offers far more flexibility. 


In [1]:
import numpy as np
import csv
import PIL

In [2]:
## Define the colour science functions

# Planck adjusted c2
# Changes as late as 2018 affected constants in Planck's black body radiation calculations
# This affects only the CIE standard illuminant D family. Do not apply to CIE illuminants A/B/C/E/F.
# As such, slight adjustments need to be made to colour termperatures in the 1000s of K. 
# Multiply things like D65 "6500" by this value to get the more accurately adusted value. 
Planck_c2_adj = 1.438776877/1.4380

# K_to_xy takes in a colour temperature in Kelvin
# returns an array of the (x,y) co-ordinates of the white point in an array
# If you are using the CIE standard illluminant D series, adjust for Planck c2.
# Ref: http://www.brucelindbloom.com/Eqn_T_to_xy.html
def K_to_xy(T):
  if T < 7000:
      x = (-4.6070e+09/T**3)+(2.9678e+06/T**2)+(0.09911e+03/T)+0.244063
  else:
      x = (-2.0064e+09/T**3)+(1.9018e+6/T**2)+(0.24748e+03/T)+0.237040
  y = (-3.000*x*x)+(2.870*x)-0.275
  return np.array([x, y])

# xy_to_XYZ takes in a normalised Y=1 xyY (x,y) co-ordinate
# returns a CIE 1931 XYZ co-ordinate array
def xy_to_XYZ(coord):
    X = coord[0]/coord[1]
    Y = 1
    Z = (1-coord[0]-coord[1])/coord[1]
    return(np.array([X, Y, Z]))

# RGB_to_XYZ_matrix takes in two arrays:
# The first is the (x,y) co-ordinates of the R, G and B chromaticities .
# The second is the XYZ co-ordinate of the white point.
# It returns a 3x3 matrix that you can then apply to a linear RGB (no gamma/EOTF) pixel to convert it to XYZ respective colourspace.
# This is referrred to as "Msrc" above.
# Ref: http://www.brucelindbloom.com/Eqn_RGB_XYZ_Matrix.html
def RGB_to_XYZ_matrix(RGBxy, white_XYZ):
    xr = RGBxy[0]
    yr = RGBxy[1]
    xg = RGBxy[2]
    yg = RGBxy[3]
    xb = RGBxy[4]
    yb = RGBxy[5]
    Xr = xr/yr
    Yr = 1
    Zr = (1-xr-yr)/yr
    Xg = xg/yg
    Yg = 1
    Zg = (1-xg-yg)/yg
    Xb = xb/yb
    Yb = 1
    Zb = (1-xb-yb)/yb
    XYZrgb = np.array([[ Xr, Xg, Xb ],
                       [ Yr, Yg, Yb ],
                       [ Zr, Zg, Zb ]])
    XYZrgb_inv = np.linalg.inv(XYZrgb)
    white_XYZ_vert = np.reshape(white_XYZ, (3,1))
    Srgbvert = np.matmul(XYZrgb_inv, white_XYZ_vert)
    Srgb=np.reshape(Srgbvert, 3)
    return(Srgb*XYZrgb)

# To calculate an RGB_to_XYZ_matrix, perform a linear algebra matrix inversion of the RGB_to_XYZ_matrix function
# with respect to the currently configured colourspace and white point of your display/monitor/TV. 
# This will give you "Mdst" above.


In [3]:
# Create list of inputs
# Commented out inputs are identical to another spec. Noting for reference, but leaving out to reduce compute.
# Comment descriptions are for refernce only.  All of these values expect linear RGB (gamma = 1, i.e.: gamma removed)
# Some XYZ_to_blah destination matrices are skipped to avoid people confusing their use case.

# Common standard white points, calculated
D65_Wxy = K_to_xy(6500*Planck_c2_adj)
D65_XYZ = xy_to_XYZ(D65_Wxy)

D93_Wxy = K_to_xy(9300*Planck_c2_adj)
D93_XYZ = xy_to_XYZ(D93_Wxy)

C_Wxy = K_to_xy(6774)
C_XYZ = xy_to_XYZ(C_Wxy)

# sRGB, D65, non-linear gamma 2.2
sRGB_desc = [ "sRGB", "sRGB spec", "non-linear gamma near 2.2" ]
sRGB_D93_desc = [ "sRGB_D93", "sRGB D93", "non-linear gamma near 2.2" ]
sRGB_Cxy = [ 0.6400, 0.3300, 0.3000, 0.6000, 0.1500, 0.0600 ]
sRGB_Wxy = [ 0.3127, 0.3290 ]
sRGB_XYZ = xy_to_XYZ(sRGB_Wxy)

sRGB_to_XYZ = RGB_to_XYZ_matrix(sRGB_Cxy, sRGB_XYZ)
XYZ_to_sRGB = np.linalg.inv(sRGB_to_XYZ)

sRGB_D93_to_XYZ = RGB_to_XYZ_matrix(sRGB_Cxy, D93_XYZ)

# BT.601-525 NTSC, D65, non-linear gamma 2.2
# Also SMPTE 170M, 240M
BT601_525_desc = [ "BT601_525", "BT.601-525 NTSC", "non-linear gamma near 2.2" ]
BT601_525_D93_desc = [ "BT601_525_D93", "BT.601-525 NTSC-J", "non-linear gamma 2.2" ]
BT601_525_Cxy = [ 0.630, 0.340, 0.310, 0.595, 0.155, 0.070 ]
#BT601_525_Wxy = sRGB_Wxy

BT601_525_to_XYZ = RGB_to_XYZ_matrix(BT601_525_Cxy, sRGB_XYZ)
BT601_525_D93_to_XYZ = RGB_to_XYZ_matrix(BT601_525_Cxy, D93_XYZ)

# BT.601-625 PAL D65, non-linear gamma 2.2
BT601_625_desc = [ "BT601_625", "BT.601-625 PAL", "non-linear gamma near 2.2" ]
BT601_625_Cxy = [ 0.640, 0.330, 0.290, 0.600, 0.150, 0.060 ]
#BT601_625_Wxy = sRGB_Wxy

BT601_625_to_XYZ = RGB_to_XYZ_matrix(BT601_625_Cxy, sRGB_XYZ)

# BT.470-6
# Also PAL-M (Brazil), SECAM (France), D65, gamma 2.8
BT470_6_desc = [ "BT460_6", "BT.470-6 / Brazil PAL-M / France SECAM", "gamma 2.8" ]
BT470_6_Cxy = [ 0.67, 0.33, 0.21, 0.71, 0.14, 0.08 ]
#BT470_6_Wxy = C_Wxy

BT470_6_to_XYZ = RGB_to_XYZ_matrix(BT470_6_Cxy, C_XYZ)

# BT.470-4 NTSC
# BT470_4_Cxy = BT470_6_Cxy

# BT.709 HD, gamma 2.4 (BT.1886, includes alt function for CRT emulation)
BT709_desc = [ "BT709", "BT.709 HD spec", "BT.1886" ]
BT709_D93_desc = [ "BT709_D93", "BT.709 HD D93", "BT.1886" ]
BT709_Cxy = [ 0.64, 0.33, 0.30, 0.60, 0.15, 0.06 ]
#BT709_Cxy = sRGB_Wxy

BT709_to_XYZ = RGB_to_XYZ_matrix(BT709_Cxy, sRGB_XYZ)
XYZ_to_BT709 = np.linalg.inv(BT709_to_XYZ)

BT709_D93_to_XYZ = RGB_to_XYZ_matrix(BT709_Cxy, D93_XYZ)

# BT.2020 UHD / 4K, SDR non-linear gamma 2.4 (BT.1886), HDR EOTF PQ / SMPTE ST 2084
BT2020_desc = [ "BT2020", "BT.2020 UHD spec", "SDR BT.1886 / HDR PQ" ]
BT2020_Cxy = [ 0.708, 0.292, 0.170, 0.797, 0.131, 0.046 ]
#BT2020_Wxy = sRGB_Wxy

BT2020_to_XYZ = RGB_to_XYZ_matrix(BT2020_Cxy, sRGB_XYZ)
XYZ_to_BT2020 = np.linalg.inv(BT2020_to_XYZ)

# BT.2100 UHD / 4K / 8K, HDR only, EOTF PQ / SMPTE ST 2084
#BT2100 Cxy = BT2020_Cxy
#BT2100_Wxy = sRGB_Wxy

# AppleRGB for pre-iMac Macintish, gamma 1.8
AppleRGB_desc = [ "AppleRGB", "AppleRGB Macintosh spec", "gamma 1.8" ]
AppleRGB_Cxy = [ 0.625 , 0.34, 0.28, 0.595, 0.155, 0.07 ]
#AppleRGB_Wxy = D65_Wxy

AppleRGB_to_XYZ = RGB_to_XYZ_matrix(AppleRGB_Cxy, D65_XYZ)
#XYZ_to_AppleRGB = np.linalg.inv(AppleRGB_to_XYZ)
XYZ_to_AppleRGB = []

# ARIB TR B9 v1.0 1125/60 HDTV
# "Japan Specific Phosphor"
ARIBTRB9v1_desc = [ "ARIBTRB9v1", "ARIB TR B9 v1.0 1125/60 HDTV", "non-linear gamma near 2.2" ]
ARIBTRB9v1_Cxy = [ 0.670, 0.330, 0.210, 0.710, 0.140, 0.080 ]
#ARIBTRB9v1_Wxy = D93_Wxy

ARIBTRB9v1_to_XYZ = RGB_to_XYZ_matrix(ARIBTRB9v1_Cxy, D93_XYZ)

# Measured Sony PVM-20M2U provide by Keith Raney @khmr33
# Gamma 2.25
Raney_PVM_20M2U_desc = [ "Raney_PVM_20M2U", "Sony PVM 20M2U by Keith Raney", "gamma 2.25" ]
Raney_PVM_20M2U_Cxy = [ 0.63, 0.345, 0.285, 0.605, 0.15, 0.065 ]
#Raney_PVM_20M2U_Wxy = D93_Wxy

Raney_PVM_20M2U_to_XYZ = RGB_to_XYZ_matrix(Raney_PVM_20M2U_Cxy, D93_XYZ)

# Measured Sony PVM-20L2MDU provide by Keith Raney @khmr33 
Raney_PVM_20L2MDU_desc = [ "Raney_PVM_20L2MDU", "Sony PVM 20L2MDU by Keith Raney", "gamma 2.25" ]
Raney_PVM_20L2MDU_Cxy = [ 0.625, 0.345, 0.28, 0.605, 0.15, 0.065 ]
#Raney_PVM_20L2MDU_Wxy = D93_Wxy

Raney_PVM_20L2MDU_to_XYZ = RGB_to_XYZ_matrix(Raney_PVM_20L2MDU_Cxy, D93_XYZ)

#print(D65_Wxy)
#print(sRGB_Wxy)
#print(D93_Wxy)

print(Raney_PVM_20L2MDU_to_XYZ)
               

[[0.39869553 0.31245042 0.24185535]
 [0.22007993 0.67511608 0.10480398]
 [0.01913739 0.12832785 1.26570966]]


In [4]:
# Write the output matrices and CSV file

csv_header = [ "col_id", "col_desc", "eotf",
               "Wx", "Wy", "Rx", "Ry", "Gx", "Gy", "Bx", "By",
               "Msrc0", "Msrc1", "Msrc2",
               "Msrc3", "Msrc4", "Msrc5",
               "Msrc6", "Msrc7", "Msrc8",
               "Mdst0", "Mdst1", "Mdst2",
               "Mdst3", "Mdst4", "Mdst5",
               "Mdst6", "Mdst7", "Mdst8" ]

csv_content = []
csv_content.append(csv_header)

csv_content.append(np.concatenate((sRGB_desc, sRGB_Wxy, sRGB_Cxy, sRGB_to_XYZ, XYZ_to_sRGB), axis=None))
csv_content.append(np.concatenate((sRGB_D93_desc, D93_Wxy, sRGB_Cxy, sRGB_D93_to_XYZ), axis=None))
csv_content.append(np.concatenate((BT601_525_desc, sRGB_Wxy, BT601_525_Cxy, BT601_525_to_XYZ), axis=None))
csv_content.append(np.concatenate((BT601_525_D93_desc, D93_Wxy, BT601_525_Cxy, BT601_525_D93_to_XYZ), axis=None))
csv_content.append(np.concatenate((BT601_625_desc, sRGB_Wxy, BT601_625_Cxy, BT601_625_to_XYZ), axis=None))
csv_content.append(np.concatenate((BT709_desc, sRGB_Wxy, BT709_Cxy, BT709_to_XYZ, XYZ_to_BT709), axis=None))
csv_content.append(np.concatenate((BT709_D93_desc, D93_Wxy, BT709_Cxy, BT709_D93_to_XYZ), axis=None))
csv_content.append(np.concatenate((BT2020_desc, sRGB_Wxy, BT2020_Cxy, BT2020_to_XYZ, XYZ_to_BT2020), axis=None))
csv_content.append(np.concatenate((AppleRGB_desc, sRGB_Wxy, AppleRGB_Cxy, AppleRGB_to_XYZ), axis=None))
csv_content.append(np.concatenate((ARIBTRB9v1_desc, D93_Wxy, ARIBTRB9v1_Cxy, ARIBTRB9v1_to_XYZ), axis=None))
csv_content.append(np.concatenate((Raney_PVM_20M2U_desc, D93_Wxy, Raney_PVM_20M2U_Cxy, Raney_PVM_20M2U_to_XYZ), axis=None))
csv_content.append(np.concatenate((Raney_PVM_20L2MDU_desc, D93_Wxy, Raney_PVM_20L2MDU_Cxy, Raney_PVM_20L2MDU_to_XYZ), axis=None))


with open('../csv/matrix.csv', 'w', newline="") as f:
    writer = csv.writer(f)
    writer.writerows(csv_content)

print("CSV output complete \n")
print("Look for the file matrix.csv in the csv folder \n")


CSV output complete 

Look for the file matrix.csv in the csv folder 

